<a href="https://colab.research.google.com/github/vedicarao/ANN-for-LBW-cases-/blob/main/MI_Assignment_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("LBW_Dataset_clean.csv")

In [3]:
df = df.drop(['Education'], axis=1)
#df = df.drop(['Residence'], axis=1)

In [4]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
X_train.shape

(76, 8)

In [8]:
# train=df.sample(frac=0.8,random_state=50) #random state is a seed value
# test=df.drop(train.index)

In [9]:
# x_train = train.iloc[:, :-1].values
# y_train = train.iloc[:, -1].values
# x_test = test.iloc[:, :-1].values
# y_test = test.iloc[:, -1].values

In [10]:
x_train_reshaped=X_train.reshape((8,76))
y_train_reshaped=y_train.reshape((1,76))

In [11]:
import numpy as np

# define activation functions g(Z), and their first derivative, using numpy
# hold all activation function pairs in a global dict
logistic = lambda Z: 1/(1+np.exp(-Z.clip(-708, 709)))
def logisticprime(Z):
    s = logistic(Z)
    return 2 * s * (1-s)

relu = lambda Z: np.maximum(0, Z)
reluprime = lambda Z: (Z > 0).astype(float)

tanh = lambda Z: np.tanh(Z)
tanhprime = lambda Z: logisticprime(2*Z) * 2

elu = lambda Z: np.where(Z>0, Z, np.exp(Z)-1)
eluprime = lambda Z: np.where(Z>0, 1, np.exp(Z))

ACTFUNC = { #    function  differentiation
    'logistic': (logistic, logisticprime),
    'relu':     (relu,     reluprime),
    'tanh':     (tanh,     tanhprime),
    'elu':      (elu,      eluprime),
}

# Loss functions L(Y, Y_hat) and their partial derivative
def xentropy(Y, Y_hat):
    """Binary cross entropy function
        L = - Y log Y_hat - (1-Y) log (1-Y_hat)
    Args:
        Y, Y_hat (np.array): nxm matrices which n are the number of data instances and m the number
                of output perceptons
    """
    eps = np.finfo(float).eps
    return -(np.dot(Y, np.log(Y_hat.clip(eps)).T) + np.dot(1-Y, np.log((1-Y_hat).clip(eps)).T)) / Y.shape[1]
def xentropyprime(Y, Y_hat):
    """ dL/dY_hat """
    eps = np.finfo(float).eps
    return - np.divide(Y, Y_hat.clip(eps)) + np.divide(1-Y, (1-Y_hat).clip(eps))

LOSSFUNC = {
        'xentropy': (xentropy, xentropyprime)
}

class pyann:
    '''Artificial Neural Network using numpy
    '''
    def __init__(self, layersizes, activations, lossfunc='xentropy'):
        """remember config, then initialize array to hold NN parameters without init"""
        # hold NN config
        self.layersizes = tuple(layersizes)
        self.activations = tuple(activations)
        self.lossfunc = lossfunc
        assert len(self.layersizes)-1 == len(self.activations), \
            "NN number of layers and the activation function spec does not match"
        assert all(f in ACTFUNC for f in activations), "Unrecognized activation function used"
        assert all(isinstance(n, int) and n >= 1 for n in layersizes), \
            "Only positive integral number of perceptons is allowed in each layer"
        assert lossfunc in LOSSFUNC, \
            "Unrecognized loss function used"
        # parameters, each is a 2D numpy array
        L = len(self.layersizes)
        self.Z = [None] * L
        self.W = [None] * L
        self.b = [None] * L
        self.A = [None] * L
        self.dZ = [None] * L
        self.dW = [None] * L
        self.db = [None] * L
        self.dA = [None] * L

    def init_nn(self, seed=35):
        """init the value of weight matrices and bias vectors with small random numbers. We do not
        use real truncated normal but a plain normal with clip at 6 sigmas. We assume we use
        activation functions that will have large derivatives around 0 so init value concentrated
        around 0 will speed up learning
        """
        np.random.seed(seed)
        sigma = 0.1
        for l, (insize, outsize) in enumerate(zip(self.layersizes, self.layersizes[1:]), 1):
            self.W[l] = np.random.randn(outsize, insize).clip(-6, 6) * sigma
            self.b[l] = np.random.randn(outsize, 1).clip(-6, 6) * sigma

    def forward(self, X):
        """Feed forward the NN using existing W and b, and overwrite the result variables A and Z
        Args:
            X (numpy.ndarray): Input data to feed forward
        """
        self.A[0] = X
        for l, funcname in enumerate(self.activations, 1):
            # Z = W A + b, with A as output from previous layer
            # W is of size rxs and A the size sxn with n the number of data instances, Z the size rxn
            # b is rx1 and broadcast to each column of Z
            g = ACTFUNC[funcname][0]
            self.Z[l] = np.dot(self.W[l], self.A[l-1]) + self.b[l]
            # A = g(Z), with A as output of this layer, of size rxn
            self.A[l] = g(self.Z[l])
        return self.A[-1]

    def backward(self, Y, Y_hat):
        """back propagation using NN output Y_hat and the reference output Y, generates dW, dZ, db,
        dA
        """
        assert Y.shape[0] == self.layersizes[-1], "Output size mismatch NN"
        assert Y.shape == Y_hat.shape, "Output size mismatch reference"
        # first dA, at the output
        self.dA[-1] = LOSSFUNC[self.lossfunc][1](Y, Y_hat)
        for l, funcname in reversed(list(enumerate(self.activations, 1))):
            m = self.layersizes[l]
            g_prime = ACTFUNC[funcname][1]
            # compute the differentials at this layer
            self.dZ[l] = self.dA[l] * g_prime(self.Z[l])
            self.dW[l] = np.dot(self.dZ[l], self.A[l-1].T) / m
            self.db[l] = np.sum(self.dZ[l], axis=1, keepdims=True) / m
            self.dA[l-1] = np.dot(self.W[l].T, self.dZ[l])

    def update(self, alpha):
        """Updates W and b
        Args:
            alpha (float): Learning rate
        """
        for l in range(1, len(self.W)):
            self.W[l] -= alpha * self.dW[l]
            self.b[l] -= alpha * self.db[l]

        # for t in range(1, len(self.W)):
        #     g = compute_gradient(x, y)
        #     m = beta_1 * m + (1 - beta_1) * g
        #     v = beta_2 * v + (1 - beta_2) * np.power(g, 2)
        #     m_hat = m / (1 - np.power(beta_1, t)) + (1 - beta_1) * g / (1 - np.power(beta_1, t))
        #     v_hat = v / (1 - np.power(beta_2, t))
        #     self.W = W - step_size * m_hat / (np.sqrt(v_hat) + epsilon)
                     s["dW" + str(l)] = ...
                    s["db" + str(l)] = ...
    def initialize_adam(self):
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    # Initialize v, s. Input: "parameters". Outputs: "v, s".
    for l in range(L):
        v["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
        v["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
        s["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
        s["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
    
    return v, s
    
    def convert_prob_into_class(self, probs):
        probs_ = np.copy(probs)
        probs_[probs_ > 0.5] = 1
        probs_[probs_ <= 0.5] = 0
        return probs_

    def get_accuracy_value(self, Y_hat, Y):
        Y_hat_ = self.convert_prob_into_class(Y_hat)
        return (Y_hat_ == Y).all(axis=0).mean()

    def fit(self, X, Y, epochs, alpha, printfreq=1):
        """Train a NN
        Args:
            X: input data, of size mxn which n is the number of data instances and m the number of
                features
            Y: reference output, of size nxm which n is the number of data instances and m the size
                of each output
            alpha: the learning rate
            epochs:
        """
        self.init_nn()
        lossfunc = LOSSFUNC[self.lossfunc][0]
        # train for each epoch
        for j in range(epochs):
            self.forward(X)
            Y_hat = self.A[-1]
            self.backward(Y, Y_hat)
            self.update(alpha)
            if printfreq and j % printfreq == 0:
                loss = float(lossfunc(Y, Y_hat))
                accuracy = self.get_accuracy_value(Y_hat, Y)
                print("Iteration {} - loss value {} - acc value {}".format(j, loss, accuracy))
        # report loss value
        accuracy = self.get_accuracy_value(Y_hat, Y)
        return lossfunc(Y, Y_hat), accuracy

      def predict

    


In [12]:
ann=pyann([8,4,6,6,6,4,1],['relu', 'relu','relu','relu','relu','logistic'])

In [13]:
ann.fit(x_train_reshaped,y_train_reshaped,20,0.01)

Iteration 0 - loss value 0.6804881934931959 - acc value 0.7631578947368421
Iteration 1 - loss value 0.6028405490018381 - acc value 0.7631578947368421
Iteration 2 - loss value 0.5719839494383603 - acc value 0.7631578947368421
Iteration 3 - loss value 0.5588704378781477 - acc value 0.7631578947368421
Iteration 4 - loss value 0.5529506433082215 - acc value 0.7631578947368421
Iteration 5 - loss value 0.5501567503354889 - acc value 0.7631578947368421
Iteration 6 - loss value 0.5487960449772237 - acc value 0.7631578947368421
Iteration 7 - loss value 0.5481185332160381 - acc value 0.7631578947368421
Iteration 8 - loss value 0.5477758877552282 - acc value 0.7631578947368421
Iteration 9 - loss value 0.547600668248748 - acc value 0.7631578947368421
Iteration 10 - loss value 0.5475103550754995 - acc value 0.7631578947368421
Iteration 11 - loss value 0.5474635408495184 - acc value 0.7631578947368421
Iteration 12 - loss value 0.5474391755803993 - acc value 0.7631578947368421
Iteration 13 - loss val

(array([[0.54741276]]), 0.7631578947368421)

In [15]:

print(ann.fit(x_train_reshaped,y_train_reshaped,200,0.001))

Iteration 0 - loss value 0.6804881934931959 - acc value 0.7631578947368421
Iteration 1 - loss value 0.6711053146647066 - acc value 0.7631578947368421
Iteration 2 - loss value 0.6624209102441657 - acc value 0.7631578947368421
Iteration 3 - loss value 0.654381861469977 - acc value 0.7631578947368421
Iteration 4 - loss value 0.6469388341347722 - acc value 0.7631578947368421
Iteration 5 - loss value 0.6400460930849733 - acc value 0.7631578947368421
Iteration 6 - loss value 0.6336613057319544 - acc value 0.7631578947368421
Iteration 7 - loss value 0.627745340271515 - acc value 0.7631578947368421
Iteration 8 - loss value 0.6222620631177208 - acc value 0.7631578947368421
Iteration 9 - loss value 0.6171781390339193 - acc value 0.7631578947368421
Iteration 10 - loss value 0.6124628365800653 - acc value 0.7631578947368421
Iteration 11 - loss value 0.6080878407784361 - acc value 0.7631578947368421
Iteration 12 - loss value 0.6040270743137806 - acc value 0.7631578947368421
Iteration 13 - loss valu